<a href="https://colab.research.google.com/github/TanyaGoremykina/Recsystems/blob/main/cw_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ПОДГОТОВКА ДАННЫХ

In [ ]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 37.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606550 sha256=ad03ef250202d9d66063df91cc61fdbb41c942633eaa8447317636a6eb59d3da
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [ ]:
! pip install LightFM

     |████████████████████████████████| 310 kB 33.5 MB/s 
  Created wheel for LightFM: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705354 sha256=b28b8e651768a7423ee9b962fe6caf66ec481ec0c1e845ab28758de8d750313e
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built LightFM


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
src_path = os.path.join(module_path, "src")
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(src_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [ ]:
from src.recommenders import MainRecommender

In [ ]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

**retail_train**

Таблица взаимодействий. Клиенты, заказы, товары, количество, цены, время, скидки

In [ ]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1.0,1004906.0,1.0,1.39,364.0,-0.60,1631.0,1.0,0.0,0.0
1,2375,26984851472,1.0,1033142.0,1.0,0.82,364.0,0.00,1631.0,1.0,0.0,0.0
2,2375,26984851472,1.0,1036325.0,1.0,0.99,364.0,-0.30,1631.0,1.0,0.0,0.0
3,2375,26984851472,1.0,1082185.0,1.0,1.21,364.0,0.00,1631.0,1.0,0.0,0.0
4,2375,26984851472,1.0,8160430.0,1.0,1.50,364.0,-0.39,1631.0,1.0,0.0,0.0


In [ ]:
data.tail()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
143597,908,28110745181,95.0,967762.0,1.0,0.89,406.0,0.00,2119.0,14.0,0.0,0.0
143598,908,28110745181,95.0,971078.0,2.0,4.00,406.0,-0.78,2119.0,14.0,0.0,0.0
143599,908,28110745181,95.0,1009128.0,1.0,1.79,406.0,0.00,2119.0,14.0,0.0,0.0
143600,908,28110745181,95.0,1039348.0,2.0,4.00,406.0,-0.98,2119.0,14.0,0.0,0.0
143601,1291,28110750453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143602 entries, 0 to 143601
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   user_id            143602 non-null  int64  
 1   basket_id          143602 non-null  int64  
 2   day                143601 non-null  float64
 3   item_id            143601 non-null  float64
 4   quantity           143601 non-null  float64
 5   sales_value        143601 non-null  float64
 6   store_id           143601 non-null  float64
 7   retail_disc        143601 non-null  float64
 8   trans_time         143601 non-null  float64
 9   week_no            143601 non-null  float64
 10  coupon_disc        143601 non-null  float64
 11  coupon_match_disc  143601 non-null  float64
dtypes: float64(10), int64(2)
memory usage: 13.1 MB


Числовые значения, пропусков нет, категориальных нет (есть числовые идентификаторы клиентов, заказов, товаров).

In [ ]:
data.describe()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
count,143602.000000,1.436020e+05,143601.000000,1.436010e+05,143601.000000,143601.000000,143601.000000,143601.000000,143601.000000,143601.000000,143601.000000,143601.000000
mean,1272.949973,2.771454e+10,61.624348,2.084514e+06,59.101678,2.982365,2477.488792,-0.567646,1601.484126,9.493367,-0.011226,-0.001990
std,732.458522,2.909231e+08,24.217352,2.530045e+06,861.603549,3.662724,7808.665022,1.153765,410.436685,3.476325,0.182483,0.031894
min,1.000000,2.698485e+10,1.000000,2.669100e+04,0.000000,0.000000,1.000000,-75.000000,0.000000,1.000000,-16.990000,-2.700000
25%,641.000000,2.750532e+10,44.000000,9.098940e+05,1.000000,1.290000,327.000000,-0.790000,1327.000000,7.000000,0.000000,0.000000
50%,1248.000000,2.777423e+10,67.000000,1.013321e+06,1.000000,2.000000,367.000000,-0.140000,1640.000000,10.000000,0.000000,0.000000
75%,1937.000000,2.795069e+10,82.000000,1.110843e+06,1.000000,3.490000,412.000000,0.000000,1915.000000,12.000000,0.000000,0.000000
max,2500.000000,2.811075e+10,96.000000,1.215964e+07,39365.000000,400.000000,32004.000000,0.000000,2359.000000,14.000000,0.000000,0.000000


**product**

Описание товаров. Идентификаторы товаров, производителей, подразделение на отделы, локальность, описание

In [ ]:
item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [ ]:
item_features.tail()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92352,18316298,764,GROCERY,National,PAPER TOWELS,PAPER TOWELS & HOLDERS,


In [ ]:
item_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92353 entries, 0 to 92352
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   item_id               92353 non-null  int64 
 1   manufacturer          92353 non-null  int64 
 2   department            92353 non-null  object
 3   brand                 92353 non-null  object
 4   commodity_desc        92353 non-null  object
 5   sub_commodity_desc    92353 non-null  object
 6   curr_size_of_product  92353 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.9+ MB


Нет пропусков, но в строковых описаниях в столбце "urr_size_of_product" есть пустые строки

In [ ]:
item_features.describe()

,item_id,manufacturer
count,9.235300e+04,92353.00000
mean,5.328353e+06,1739.22833
std,5.359937e+06,1818.26957
min,2.567100e+04,1.00000
25%,9.706280e+05,328.00000
50%,1.621091e+06,1094.00000
75%,9.704770e+06,2264.00000
max,1.831630e+07,6477.00000


In [ ]:
def print_categorical(data):
    for cat_colname in data.columns:
        if data[cat_colname].nunique() < 310:
            print(str(cat_colname) + '\n\n' + str(data[cat_colname].unique()) + '\n\n')
            print(data.groupby(cat_colname)[cat_colname].count())
            print('*' * 100 + '\n')

In [ ]:
print_categorical(item_features)

department

['GROCERY' 'MISC. TRANS.' 'PASTRY' 'DRUG GM' 'MEAT-PCKGD' 'SEAFOOD-PCKGD'
 'PRODUCE' 'NUTRITION' 'DELI' 'COSMETICS' 'MEAT' 'FLORAL'
 'TRAVEL & LEISUR' 'SEAFOOD' 'MISC SALES TRAN' 'SALAD BAR' 'KIOSK-GAS'
 'ELECT &PLUMBING' 'GRO BAKERY' 'GM MERCH EXP' 'FROZEN GROCERY'
 'COUP/STR & MFG' 'SPIRITS' 'GARDEN CENTER' 'TOYS' 'CHARITABLE CONT'
 'RESTAURANT' 'RX' 'PROD-WHS SALES' 'MEAT-WHSE' 'DAIRY DELI' 'CHEF SHOPPE'
 'HBC' 'DELI/SNACK BAR' 'PORK' 'AUTOMOTIVE' 'VIDEO RENTAL' ' '
 'CNTRL/STORE SUP' 'HOUSEWARES' 'POSTAL CENTER' 'PHOTO' 'VIDEO'
 'PHARMACY SUPPLY']


department
                      15
AUTOMOTIVE             2
CHARITABLE CONT        2
CHEF SHOPPE           14
CNTRL/STORE SUP        4
COSMETICS           3011
COUP/STR & MFG        39
DAIRY DELI             3
DELI                2354
DELI/SNACK BAR         2
DRUG GM            31529
ELECT &PLUMBING        1
FLORAL               938
FROZEN GROCERY        23
GARDEN CENTER        128
GM MERCH EXP           3
GRO BAKERY       

In [ ]:
item_features['sub_commodity_desc'].unique()

array(['ICE - CRUSHED/CUBED', 'NO SUBCOMMODITY DESCRIPTION',
       'BREAD:ITALIAN/FRENCH', ..., 'SEMI MOIST CAT FOOD: BOX',
       'BERRIES OTHER', 'ROSES OTHER'], dtype=object)

In [ ]:
item_features['sub_commodity_desc'].nunique()

2383

In [ ]:
item_features['curr_size_of_product'].unique()

array(['22 LB', ' ', '50 OZ', ..., '6.3 IN', '35 LD', '2 LTR PET'],
      dtype=object)

In [ ]:
item_features['curr_size_of_product'].nunique()

4345

**hh_demographic**

Описание клиентов. Идентификаторы клиентов, возрастные группы, семейное положение, группы по доходу, данные по недвижимости и количеству детей

In [ ]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [ ]:
user_features.tail()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498
800,25-34,U,Under 15K,Unknown,2 Adults Kids,3,1,2499


In [ ]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   age_desc             801 non-null    object
 1   marital_status_code  801 non-null    object
 2   income_desc          801 non-null    object
 3   homeowner_desc       801 non-null    object
 4   hh_comp_desc         801 non-null    object
 5   household_size_desc  801 non-null    object
 6   kid_category_desc    801 non-null    object
 7   user_id              801 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 50.2+ KB


In [ ]:
data.user_id.nunique()

1857

In [ ]:
f"{user_features.user_id.nunique()/data.user_id.nunique()*100:.2f}%"

'43.13%'

In [ ]:
u_all = set(data.user_id.unique().tolist())
u_inf = set(user_features.user_id.unique().tolist())

len(u_all.intersection(u_inf))

580

Пропусков нет, но данные охватывают только 43.13% всех пользователей

**Фильтрация**

Фильтруем 3 самых популярных

In [ ]:
popularity = data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [ ]:
item_features.loc[item_features['item_id'].isin(top_5000[:5])]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
3565,397896,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
57181,6533889,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
57216,6534166,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
57221,6534178,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
57335,6544236,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,


In [ ]:
data.loc[data['item_id'].isin(top_5000[:3]), 'item_id'] = 999999

**Уберем товары, которые не продавались за последние 12 месяцев**

In [ ]:
curr_week = data['week_no'].values.max()
data.loc[data['week_no']<curr_week-53, 'item_id'] = 999999

**Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоят < 1$**

In [ ]:
mean_price = data.groupby('item_id')['quantity', "sales_value"].sum()
mean_price["mean_price"] = mean_price["sales_value"] / mean_price["quantity"]
cheap_list = mean_price.loc[mean_price["mean_price"]<0.9].index.tolist()
data.loc[data['item_id'].isin(cheap_list), 'item_id'] = 999999

**Cамые непопулярные**

In [ ]:
popularity_pu = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity_pu.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
popularity_pu.sort_values('share_unique_users', ascending=False, inplace=True)
popularity_pu.tail()

,item_id,share_unique_users
11046,1018943.0,1
11047,1018944.0,1
11051,1018964.0,1
11052,1018977.0,1
24365,12159641.0,1


In [ ]:
popularity_pu = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity_pu.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
popularity_pu['share_unique_users'] = popularity_pu['share_unique_users'] / data['user_id'].nunique()
popularity_pu.sort_values('share_unique_users', ascending=False, inplace=True)
popularity_pu.head()

,item_id,share_unique_users
10068,999999.0,0.833064
14263,1082185.0,0.400108
11578,1029743.0,0.199785
9779,995242.0,0.186322
9093,981760.0,0.169628


In [ ]:
popularity_pu.shape

(24366, 2)

In [ ]:
popularity_pu.tail()

,item_id,share_unique_users
11046,1018943.0,0.000539
11047,1018944.0,0.000539
11051,1018964.0,0.000539
11052,1018977.0,0.000539
24365,12159641.0,0.000539


In [ ]:
popularity_pu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24366 entries, 10068 to 24365
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item_id             24366 non-null  float64
 1   share_unique_users  24366 non-null  float64
dtypes: float64(2)
memory usage: 571.1 KB


In [ ]:
popularity_pu[popularity_pu['share_unique_users'] < 0.005]

,item_id,share_unique_users
9198,983659.0,0.004847
9982,998666.0,0.004847
2535,851129.0,0.004847
16877,1134633.0,0.004847
2831,856942.0,0.004847
...,...,...
11046,1018943.0,0.000539
11047,1018944.0,0.000539
11051,1018964.0,0.000539
11052,1018977.0,0.000539


In [ ]:
unpop_list = popularity_pu[popularity_pu['share_unique_users'] < 0.005].item_id.tolist()
data.loc[data['item_id'].isin(unpop_list), 'item_id'] = 999999

**Не интересные для рекоммендаций категории (department)**

In [ ]:
item_features.shape

(92353, 7)

In [ ]:
item_features['department'].unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [ ]:
item_features[item_features['department'] == 'PROD-WHS SALES']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
10781,862237,2,PROD-WHS SALES,National,PROD SUPPLIES,TICKETS,
30852,1040186,2,PROD-WHS SALES,National,PROD SUPPLIES,COUPON,


In [ ]:
item_features[item_features['department'] == 'RX']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
10776,862185,5983,RX,National,PHARMACY,MENSTRUAL ANALGESICS,30 CT
23890,977999,6073,RX,National,PHARMACY,JHOOK - PERSONAL CARE,
30923,1040789,5720,RX,National,PHARMACY,FIRST AID MEDICATIONS,
33043,1059903,125,RX,National,PHARMACY,N-PRESC HEARING AID BATTERIES,4 CT
42459,1185885,5,RX,Private,COUPON/MISC ITEMS,MISC SALES TRANS,
44511,1377652,2151,RX,National,PHARMACY,CONVALESCENT CARE - CANES,1 CT
52595,5565609,569,RX,National,PHARMACY,RX OTC BEAUTYASSESSPILL BX/H,
66044,9368445,5059,RX,National,PHARMACY,COLD AND FLU - DXM,40 CT
79409,12810538,851,RX,National,PHARMACY,N-PRESC HEARING AID BATTERIES,16 CT


In [ ]:
item_features[item_features['department'] == 'MEAT-WHSE']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
12746,879637,5995,MEAT-WHSE,National,COUPON,MISC SALES TRANS,


In [ ]:
item_features[item_features['department'] == 'HBC']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
21478,956174,177,HBC,National,INFANT CARE PRODUCTS,FEEDING ACCESSORIES BOTTLES,


In [ ]:
item_features[item_features['department'] == 'CNTRL/STORE SUP']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
59612,7100347,1871,CNTRL/STORE SUP,National,COUPON/MISC ITEMS,CENTRAL SUPPLIES,125 CT
78400,12671292,5879,CNTRL/STORE SUP,National,COUPON/MISC ITEMS,COFF SHOP: RETAIL PACK BEVERAG,1 LB
79932,13004709,6011,CNTRL/STORE SUP,National,COUPON/MISC ITEMS,CENTRAL SUPPLIES,
91676,17330423,903,CNTRL/STORE SUP,Private,COUPON/MISC ITEMS,ELECTRONIC GIFT CARDS ACTIVATI,


In [ ]:
item_features[item_features['department'] == 'POSTAL CENTER']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
64268,8293561,693,POSTAL CENTER,National,CANDY - PACKAGED,CANDY BAGS-CHOCOCLATE,12 PK
91025,17179638,693,POSTAL CENTER,National,CANDY - PACKAGED,SEASONAL CANDY BOX-CHOCOLATE,
92256,18143349,857,POSTAL CENTER,National,CANDY - CHECKLANE,CANDY BARS (SINGLES)(INCLUDING,


In [ ]:
item_features[item_features['department'] == 'PHOTO']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
79669,12949577,1225,PHOTO,National,REFRGRATD DOUGH PRODUCTS,REFRIGERATED COOKIES-SEASONAL,18 OZ
85781,14077294,236,PHOTO,National,REFRGRATD DOUGH PRODUCTS,MISC REFRIG DOUGH PRODUCTS,8 OZ


In [ ]:
item_features[item_features['department'] == 'VIDEO']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
85754,14073108,6375,VIDEO,National,BEERS/ALES,BEERALEMALT LIQUORS,
87652,15681643,239,VIDEO,National,BEERS/ALES,BEERALEMALT LIQUORS,


In [ ]:
item_features[item_features['department'] == 'PHARMACY SUPPLY']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
86113,15449615,6129,PHARMACY SUPPLY,National,COUPON/MISC ITEMS,JUICE,13.5 OZ


In [ ]:
dep_list = ['PROD-WHS SALES', 'RX', 'MEAT-WHSE', 
            'HBC', 'CNTRL/STORE SUP', 'POSTAL CENTER', 
            'PHOTO', 'VIDEO', 'PHARMACY SUPPLY']

In [ ]:
for dep in dep_list:
    print(item_features.loc[item_features['department'] == dep, "item_id"].values)

[ 862237 1040186]
[  862185   977999  1040789  1059903  1185885  1377652  5565609  9368445
 12810538]
[879637]
[956174]
[ 7100347 12671292 13004709 17330423]
[ 8293561 17179638 18143349]
[12949577 14077294]
[14073108 15681643]
[15449615]


In [ ]:
for dep in dep_list:
    drop_items_list = item_features.loc[item_features['department'] == dep, "item_id"].values
    sl = data.loc[data['item_id'].isin(drop_items_list), 'item_id'].count()
    print(f"dep: {dep} sale count: {sl}")
    data.loc[data['item_id'].isin(drop_items_list), 'item_id'] = 999999

dep: PROD-WHS SALES sale count: 0
dep: RX sale count: 0
dep: MEAT-WHSE sale count: 0
dep: HBC sale count: 0
dep: CNTRL/STORE SUP sale count: 0
dep: POSTAL CENTER sale count: 0
dep: PHOTO sale count: 0
dep: VIDEO sale count: 0
dep: PHARMACY SUPPLY sale count: 0


**Feature Engineering**

Средний чек

In [ ]:
bills = data.groupby(["basket_id", 'user_id'])["sales_value"].sum()
bills

basket_id    user_id
26984851472  2375        5.91
26984851516  2375       12.43
26984896261  1364       11.37
26984905972  1130        2.55
26984945254  1173        3.53
                        ...  
28110743546  1888       17.92
28110743764  40         34.01
28110744748  1845       51.53
28110745181  908        14.86
28110750453  1291        0.00
Name: sales_value, Length: 15700, dtype: float64

In [ ]:
bills = bills.reset_index(level=[0,1])
bills.head()

,basket_id,user_id,sales_value
0,26984851472,2375,5.91
1,26984851516,2375,12.43
2,26984896261,1364,11.37
3,26984905972,1130,2.55
4,26984945254,1173,3.53


In [ ]:
bills_avg = data.groupby(['user_id'])["basket_id"].count()
bills_avg

user_id
1       77
5        6
7       73
8       96
12      15
        ..
2493    90
2495     6
2497    34
2499    93
2500    30
Name: basket_id, Length: 1857, dtype: int64

In [ ]:
bills_avg = bills_avg.reset_index()
bills_avg.columns=['user_id', 'bills_cnt']
bills_avg

,user_id,bills_cnt
0,1,77
1,5,6
2,7,73
3,8,96
4,12,15
...,...,...
1852,2493,90
1853,2495,6
1854,2497,34
1855,2499,93


In [ ]:
bills_sum = bills.groupby(['user_id'])["sales_value"].sum()
bills_sum

user_id
1       210.09
5        23.70
7       218.88
8       315.70
12       46.58
         ...  
2493    245.57
2495     23.19
2497    173.18
2499    236.57
2500    111.40
Name: sales_value, Length: 1857, dtype: float64

In [ ]:
bills_avg['bill_sum'] = bills_sum.values
bills_avg.head()

,user_id,bills_cnt,bill_sum
0,1,77,210.09
1,5,6,23.70
2,7,73,218.88
3,8,96,315.70
4,12,15,46.58


In [ ]:
bills_avg['bill_avg'] = bills_avg['bill_sum'] / bills_avg['bills_cnt']
bills_avg.head()

,user_id,bills_cnt,bill_sum,bill_avg
0,1,77,210.09,2.728442
1,5,6,23.70,3.950000
2,7,73,218.88,2.998356
3,8,96,315.70,3.288542
4,12,15,46.58,3.105333


In [ ]:
user_features = user_features.merge(bills_avg[["user_id", "bill_avg"]], on='user_id', how='left')
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,bill_avg
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,2.728442
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.998356


**Когда была крайняя покупка**

In [ ]:
bills_per_week = data.groupby(['user_id'])["week_no"].max()
bills_per_week

user_id
1       14.0
5       13.0
7       14.0
8       13.0
12      11.0
        ... 
2493    14.0
2495    14.0
2497    14.0
2499    14.0
2500    14.0
Name: week_no, Length: 1857, dtype: float64

In [ ]:
bills_per_week = bills_per_week.reset_index()
bills_per_week.columns = ["user_id", "week_last_bill"]
bills_per_week

,user_id,week_last_bill
0,1,14.0
1,5,13.0
2,7,14.0
3,8,13.0
4,12,11.0
...,...,...
1852,2493,14.0
1853,2495,14.0
1854,2497,14.0
1855,2499,14.0


In [ ]:
user_features = user_features.merge(bills_per_week, on='user_id', how='left')
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,bill_avg,week_last_bill
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,2.728442,14.0
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.998356,14.0


**Количество покупок в неделю**

In [ ]:
bills_week = data.groupby(['user_id', 'week_no'])["item_id"].count()
bills_week

user_id  week_no
1        8.0        30
         10.0       12
         13.0       12
         14.0       23
5        13.0        6
                    ..
2499     11.0       10
         13.0       82
         14.0        1
2500     12.0       22
         14.0        8
Name: item_id, Length: 7554, dtype: int64

In [ ]:
bills_week = bills_week.reset_index()
bills_week.columns = ["user_id", "week_no", "item_cnt"]
bills_week.head()

,user_id,week_no,item_cnt
0,1,8.0,30
1,1,10.0,12
2,1,13.0,12
3,1,14.0,23
4,5,13.0,6


In [ ]:
bills_count = bills_week.groupby("user_id")["item_cnt"].sum()
bills_count

user_id
1       77
5        6
7       73
8       96
12      15
        ..
2493    90
2495     6
2497    34
2499    93
2500    30
Name: item_cnt, Length: 1857, dtype: int64

In [ ]:
bills_per_week['item_cnt'] = bills_count.values

In [ ]:
old_bill = data.groupby(['user_id'])["week_no"].min()
old_bill

user_id
1        8.0
5       13.0
7        4.0
8       10.0
12      11.0
        ... 
2493     3.0
2495    11.0
2497    12.0
2499    11.0
2500    12.0
Name: week_no, Length: 1857, dtype: float64

In [ ]:
bills_per_week["week_oldest_bill"] = old_bill.values
bills_per_week.head()

,user_id,week_last_bill,item_cnt,week_oldest_bill
0,1,14.0,77,8.0
1,5,13.0,6,13.0
2,7,14.0,73,4.0
3,8,13.0,96,10.0
4,12,11.0,15,11.0


In [ ]:
bills_per_week["user_int"] = 95 - bills_per_week["week_oldest_bill"]
bills_per_week

,user_id,week_last_bill,item_cnt,week_oldest_bill,user_int
0,1,14.0,77,8.0,87.0
1,5,13.0,6,13.0,82.0
2,7,14.0,73,4.0,91.0
3,8,13.0,96,10.0,85.0
4,12,11.0,15,11.0,84.0
...,...,...,...,...,...
1852,2493,14.0,90,3.0,92.0
1853,2495,14.0,6,11.0,84.0
1854,2497,14.0,34,12.0,83.0
1855,2499,14.0,93,11.0,84.0


In [ ]:
bills_per_week["items_per_week"] = bills_per_week["item_cnt"] / bills_per_week["user_int"]
bills_per_week.head()

,user_id,week_last_bill,item_cnt,week_oldest_bill,user_int,items_per_week
0,1,14.0,77,8.0,87.0,0.885057
1,5,13.0,6,13.0,82.0,0.073171
2,7,14.0,73,4.0,91.0,0.802198
3,8,13.0,96,10.0,85.0,1.129412
4,12,11.0,15,11.0,84.0,0.178571


In [ ]:
user_features = user_features.merge(bills_per_week[["user_id", "items_per_week"]], on='user_id', how='left')
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,bill_avg,week_last_bill,items_per_week
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,2.728442,14.0,0.885057
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.998356,14.0,0.802198


**Сохраняем предобработанные данные**

In [ ]:
data.to_csv("data.csv", index=False)
user_features.to_csv("user_features.csv", index=False)
item_features.to_csv("item_features.csv", index=False)